In [1]:
import numpy as np
from math import log2
import time
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
#===========================================#
#to use for testing, amend path to txt file
#===========================================#

clean_data = np.loadtxt("clean_dataset.txt") 
noisy_data = np.loadtxt("noisy_dataset.txt")

In [3]:
def entropy(data): # input is a multi dimensonal numpy array 
    num_entries = len(data) # total number of instances in the dataset
    outcomes = defaultdict(int) # dict to hold count of each label in dataset
    entropy = 0 # initial entropy value
    for row in data:
        key = row[-1] # label is last value of each instance
        outcomes[key] += 1 # increment the count of the label 

    for key in outcomes:
        outcomes[key] /= num_entries # compute the probability of each label 
        prob_label = outcomes[key] 
        entropy -= (prob_label*log2(prob_label)) # computing the entropy value

    return entropy

In [4]:
def InfoGain(All, Left, Right):
    
    H_SAll=entropy(All) #entropy of parent datset
    Total=len(All)
    Remainder=((len(Left)/Total)*entropy(Left))+((len(Right)/Total)*entropy(Right)) #weighted entropies of left and right datasets
    Gains=H_SAll-Remainder
    
    return Gains

In [5]:
def FIND_SPLIT(dataset):
    emitter = 0
    value = 0
    max_info_gain = 0
    right = []
    left = []
    for x in range(len(dataset[0])-1): #first 7 entries, exclude room
        
        ds = np.array(sorted(dataset, key = lambda y: y[x], reverse=True)) #sort n dimensional array by emitter
        
        for r in range(len(ds)): #iterate over every value for each emitter
            
            split_point = ds[r][x] #current point to check
            if(r!=len(ds)-1 and ds[r+1][x] == split_point): #check for repeated values, use last instance of value as split point
                continue
            info_gain = InfoGain(ds, ds[:r+1],ds[r+1:]) #calculte information gain for current point
            
            if(info_gain > max_info_gain): #if information gain is higher, update the best split point and emitter
                left = ds[:r+1]
                right = ds[r+1:]
                emitter = x
                value = split_point
                max_info_gain = info_gain
            
            
        
    #return emitter, value, left, right
    return left, right

In [6]:
def tree_learning(training_data, depth):
    
    if(entropy(training_data) == 0 and len(training_data != 0)): #if all room labels are the same, return leaf which contains room number
        count = len(training_data) #counting the number of entries in the dataset where the room labels are all the same (used later for pruning purposes)
        label = training_data[0][-1]
        leaf = {'emitter':-1, 'value':-1, 'room': label, 'right':-1, 'left':-1, 'count': count, 'leaf': True}
        return leaf, depth
    
    else: 
        em, val, ldata, rdata = FIND_SPLIT(training_data) #split data to get the child datasets and split point
        root = {'emitter': em, 'value': val, 'room': -1, 'right':-1, 'left':-1, 'count': -1, 'leaf': False} #create non leaf node
        root['left'], l_depth = tree_learning(ldata, depth+1) #recursively call tree learning to generate rest of tree, increasing depth
        root['right'], r_depth = tree_learning(rdata, depth+1)
        return root, max(l_depth, r_depth) #return root of tree and depth

In [7]:
#===========================================#
#for testing tree generation only
#===========================================#
st=time.time()
tree_root, depth = tree_learning(clean_data, 0)
et = time.time()
print(et-st)
#print(tree_root)

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
def return_result(row, root):
    
    if(root['room']!= -1): #check for leaf node, return room
        return root['room']
    else:
        if(row[root['emitter']]>=root['value']): #traverse tree to find leaf node
            return return_result(row, root['left'])
        else:
            return return_result(row, root['right'])

In [ ]:
def confusion_matrix(results_table): 
    
    cm = np.zeros((4,4)) #empty confusion matrix
    
    for i in range (1,5):
        for pair in results_table: #pair[0] is actual value, pair[1] is predicted value by decision tree
            if(i == pair[0] and pair[0] == pair[1]): #count correct predictions and update confusion matrix
                    cm[i-1][i-1]+=1
            if(i == pair[0] and pair[1] != i):
                    cm[i-1][int(pair[1])-1]+=1
    return cm

In [ ]:
def accuracy(cm):
    if(np.sum(cm) == 0):
        return 0
    return np.trace(cm)/np.sum(cm) #returns accuracy from confusion matrix

In [ ]:
#===========================================#
#function returns BOTH confusion matrix and accuracy, different from specification which only required us to return accuracy
#===========================================#

def evaluate(test, root): 
    
    results_table = np.empty((len(test),2))
    
    for i in range(len(test)): #iterate over test data and generate prediction
        prediction = return_result(test[i], root)
        results_table[i] = [test[i][-1],prediction] #keep track of actual outcome vs predicted outcome
        
    cm = confusion_matrix(results_table) #generate confusion matrix from results table
    tree_accuracy = accuracy(cm) #calculate accuracy from confusion matrix
    
    return cm, tree_accuracy 

In [ ]:
#===========================================#
#cross validation WITHOUT pruning
#===========================================#

def cross_validation(dataset):

    np.random.shuffle(dataset) #shuffles dataset

    fold_size = int(len(dataset)/10) 
    running_cm = np.zeros((4,4)) #generate empty confusion matrix
    
    for i in range(10):
        
        #cycling test and training data for cross valdiation
        test = dataset[0+fold_size*i:fold_size*(i+1)] 
        training = np.vstack((dataset[0:0+fold_size*i],dataset[fold_size*(i+1):])) 
        
        root, depth = tree_learning(training, 0) #generate tree for each cycled training dataset
        
        confusion_matrix, tree_accuracy = evaluate(test, root) #generate confusion matrix for current tree   
        running_cm += confusion_matrix #keep track of confusion matrices to calculate average
            
    avg_cm = running_cm/10 #calculate average confusion matrix
    
    return avg_cm #return average confusion matrix
    
    
    

In [ ]:
#-----------------------------------------#
testing = np.loadtxt("clean_dataset.txt")
st=time.time()
print(cross_validation(testing))
et=time.time()
print('Runtime: ', et-st)
#-----------------------------------------#

In [ ]:
def precision(cm): 
    class_precisions = np.empty(4)
    cols = cm.sum(axis=0) 
    
    for i in range(len(cm)):
        class_precisions[i] = (cm[i][i]/cols[i]) #calculate precision for each class
    
    return class_precisions

In [ ]:
def recall(cm):
    class_recalls = np.empty(4)
    rows = cm.sum(axis=1)
   
    for i in range(len(cm)):
        class_recalls[i] = (cm[i][i]/rows[i]) #calculate recall for each class
    
    return class_recalls

In [ ]:
def f1(recall, precision):
    class_f1s = np.empty(4)
    for i in range(len(recall)):
        class_f1s[i] = ((2*recall[i]*precision[i])/(recall[i]+precision[i])) #calculate f1 measure for each class using precision and recall values
    
    return class_f1s
    

In [ ]:
def prune(root, validation_set, start): 
        
        
    if ((root['left']['emitter'] == -1) and (root['right']['emitter'] == -1)): #if both children leaf nodes
        
        old = [root['emitter'], root['value'], root['room'], root['right'], root['left'], root['count']] #store all parent and children before pruning

        cm, old_accuracy = evaluate(validation_set, start) #obtain accuracy of tree before prune
        
        
        if(root['left']['count'] > root['right']['count']): #choose majority class label using count value
            root['room'] = root['left']['room']
            root['count'] = root['left']['count']
        else:
            root['room'] = root['right']['room']
            root['count'] = root['right']['count']

        #make parent node into single leaf node
        root['left'] = -1 
        root['right'] = -1
        root['emitter'] = -1
        root['value'] = -1
        root['leaf'] = True
        
        
        cm, new_accuracy = evaluate(validation_set, start) #obtain accuracy of tree after pruning
        
        if new_accuracy < old_accuracy: #if pre-pruned accuracy was better, undo prune
            root['emitter'] = old[0]
            root['value'] = old[1]
            root['room'] = old[2]
            root['right'] = old[3]
            root['left'] = old[4]
            root['count'] = old[5]
        
        return start
            
    else:
        
        #traverse tree to find prunable nodes
        if(root['left']['emitter'] != -1): 
            start = prune(root['left'], validation_set, start)

        if(root['right']['emitter'] != -1):
            start = prune(root['right'], validation_set, start)
            
    return start

In [ ]:
#===========================================#
#cross validation WITH pruning
#===========================================#

def cross_validation_prune(dataset):

    np.random.shuffle(dataset) #shuffle dataset
    fold_size = int(len(dataset)/10)
    
    #initialise confusion matrices for pre and post pruned trees
    running_cm = np.zeros((4,4))
    running_cm_pr = np.zeros((4,4))
    worse = False
    
    for i in range(10):
        
        #cycle test, training and validation sets
        test = dataset[fold_size*i:fold_size*(i+1)]
        validation = dataset[fold_size*(i+1)+1:fold_size*(i+2)+1]
        training = np.vstack((dataset[0:fold_size*i],dataset[fold_size*(i+2)+1:]))
        
        #generate decision tree
        root, depth = tree_learning(training, 0)
                
        confusion_matrix, tree_accuracy = evaluate(test, root) #generate confusion matrix for current tree   
        running_cm += confusion_matrix #keep track of confusion matrices to calculate average
        old_acc = tree_accuracy
        while(True): #keep pruning until accuracy does not improve
            pruned_tree = prune(root, validation, root) #generate pruned tree
            x, cur_acc = evaluate(validation, root)
            if(cur_acc <= old_acc):
                break
            old_acc = cur_acc
            
        #print('Depth after:', findDepth(pruned_tree))
        
        cm_prune, pr_acc = evaluate(test, pruned_tree) #generate confusion matrix for current pruned tree 
        running_cm_pr += cm_prune #keep track of confusion matrices to calculate average for pruned tree
        
        print(depth, " vs ", findDepth(pruned_tree))
        
#       print(tree_accuracy, " vs ", pr_acc)
        
    
    avg_cm = running_cm/10
    avg_cm_pr = running_cm_pr/10
    
#   print("Before prune accuracy:", accuracy(avg_cm))
#   print("After prune accuracy:", accuracy(avg_cm_pr))
    
    return accuracy(avg_cm), accuracy(avg_cm_pr)
    

In [ ]:
#===========================================#
#nested cross validation WITH pruning
#===========================================#

def nested_cross_validation_prune(dataset):

    np.random.shuffle(dataset) #shuffle dataset
    fold_size = int(len(dataset)/10)
    
    #initialise confusion matrices for pre and post pruned trees
    running_cm = np.zeros((4,4))
    running_cm_pr = np.zeros((4,4))
    
    for j in range(10):
    
        test = dataset[fold_size*j:fold_size*(j+1)]
        traning_validation_dataset = np.vstack((dataset[0:fold_size*j], dataset[fold_size*(j+1):]))
        
        for i in range(9):
        
            #cycle test, training and validation sets
            
            validation = traning_validation_dataset[fold_size*(i+1)+1:fold_size*(i+2)+1]
            training = np.vstack((traning_validation_dataset[0:fold_size*i],traning_validation_dataset[fold_size*(i+2)+1:]))
        
            #generate decision tree
            root, depth = tree_learning(training, 0)
                
            confusion_matrix, tree_accuracy = evaluate(test, root) #generate confusion matrix for current tree   
            running_cm += confusion_matrix #keep track of confusion matrices to calculate average
                  
            old_acc = tree_accuracy
            while(True): #keep pruning until accuracy does not improve
                pruned_tree = prune(root, validation, root) #generate pruned tree
                x, cur_acc = evaluate(validation, root)
                if(cur_acc <= old_acc):
                    break
                old_acc = cur_acc
            #print('Depth after:', findDepth(pruned_tree))
        
            cm_prune, pr_acc = evaluate(test, pruned_tree) #generate confusion matrix for current pruned tree 
            running_cm_pr += cm_prune #keep track of confusion matrices to calculate average for pruned tree
        
            #print(depth, " vs ", findDepth(pruned_tree))
            #print(tree_accuracy, " vs ", pr_acc)

    
    avg_cm = running_cm/90
    avg_cm_pr = running_cm_pr/90
    
#   print("Before prune accuracy:", accuracy(avg_cm))
#   print("After prune accuracy:", accuracy(avg_cm_pr))
    
    return accuracy(avg_cm), accuracy(avg_cm_pr)
    

In [ ]:
#-----------------------------------------#
testing = np.loadtxt("clean_dataset.txt")
st=time.time()
matrix = cross_validation(testing)
print("Clean Matrix: ", matrix)
acc = accuracy(matrix)
pres = precision(matrix)
rec = recall(matrix)
print("Clean accuracy: ", acc)
print("Clean precision: ", pres)
print("Clean recall: ", rec)
fmeasure = f1(rec, pres)
print("Clean fmeasure: ", fmeasure )
et=time.time()
print('Runtime: ', et-st)
#-----------------------------------------#

In [ ]:
#-----------------------------------------#
testing = np.loadtxt("noisy_dataset.txt")
st=time.time()
matrix = cross_validation(testing)
print("noisy Matrix: ", matrix)
acc = accuracy(matrix)
pres = precision(matrix)
rec = recall(matrix)
print("noisy accuracy: ", acc)
print("noisy precision: ", pres)
print("noisy recall: ", rec)
fmeasure = f1(rec, pres)
print("noisy fmeasure: ", fmeasure )
et=time.time()
print('Runtime: ', et-st)
#-----------------------------------------#

In [ ]:
def findDepth(node): 
    
    lDepth = 0
    rDepth = 0
    
    if(node['leaf'] ==  True): 
        return 0
  
    else : 

        #find depth for each subtree
        lDepth = findDepth(node['left']) 
        rDepth = findDepth(node['right']) 
  
        #take the larger depth since looking for maximum
        if (lDepth > rDepth): 
            return lDepth+1
        else: 
            return rDepth+1

In [ ]:
#print
def countLeaf(tree):
    if(tree is None):
        return 0
    if(tree['left'] == -1 and tree['right'] == -1): 
        return 1
    else:
        return countLeaf(tree['left']) + countLeaf(tree['right'])

leafNode=dict(boxstyle="round4", fc=(1., .8, 1.))
treeNode=dict(boxstyle="round4", fc="0.8") #style of the bubble around the nodes
arrows=dict(arrowstyle="<-") #type of arrows 

##################
#function to draw the node bubble with the text
##################
def draw_node(nodeText,centerPt,parentPt,nodeType):
    display_tree.ax1.annotate(nodeText,xy=parentPt,xycoords='axes fraction',xytext=centerPt, va='bottom',ha='center',bbox=nodeType,arrowprops=arrows)

def draw_decision(cntrPt, parentPt, decision):
    xPt = (parentPt[0]-cntrPt[0])/2.0+cntrPt[0]
    yPt = (parentPt[1]-cntrPt[1])/2.0+cntrPt[1]
    if(decision is True):
        display_tree.ax1.text(xPt, yPt, "True")
    else:
        display_tree.ax1.text(xPt, yPt, "False")
        
def draw_tree(root, parentPt, decision):
    if (root is None):
        return
    leaf_num = countLeaf(root)  
    height = findDepth(root)    
    cntrPt = (draw_tree.x + (1.0 + float(leaf_num))/2.0/draw_tree.total_width, draw_tree.y) #determines the x and y coordinates for the node
    draw_tree.y = (draw_tree.y - 1.0/draw_tree.total_depth)  # y coordinate for the next level of the tree
    draw_decision(cntrPt, parentPt, decision)
    #if (root['right'] != -1 and root['left'] != -1):  #for non leaf nodes
    if(root['leaf'] is False):
        textInNode=str('X <' + str(root['value'])+'\n'+ str(root['emitter']))  #text that is supposed to be in the node
        draw_node(textInNode, cntrPt, parentPt, treeNode)
        draw_tree(root['left'], cntrPt, False)
        draw_tree(root['right'], cntrPt, True)        
        
    else:   #it's a leaf node print the leaf node
        textInNode=str(root['room'])   #text that is supposed to be in the leaf node
        draw_tree.x = (draw_tree.x + 1.0/draw_tree.total_width)  # x coordinate for the new node
        draw_node(textInNode, cntrPt, parentPt, leafNode) 
    draw_tree.y = (draw_tree.y + 1.0/draw_tree.total_depth)  # y coord for the next level of the tree

def display_tree(root):
    fig=plt.figure(1,facecolor='white')  # new figure
    
    axprops=dict(xticks=[],yticks=[])  #without the x and y major and minor ticks 
    display_tree.ax1=plt.subplot(111,frameon=False,**axprops) #create a subplot without the x y axesa and without ticks 
    draw_tree.total_width=float(0.3*countLeaf(root))  #constant used in the draw_tree 
    draw_tree.total_depth=float(0.3*findDepth(root))  #
    draw_tree.x=-0.5/draw_tree.total_width  # x coordinates
    draw_tree.y=1.0  #y coordinates
    draw_tree(root,(0.5,2.0),None)
    plt.savefig('out.png', bbox_inches='tight')  #output the figure to a png
    plt.show()
    plt.close(fig)
    
clean_rows = np.loadtxt("clean_dataset.txt")
t, d = tree_learning(clean_rows, 0)

display_tree(t)